<Header> This is is where we develop a baseline factor model that allows us to take certain factors and use them in multi-OLS regression to predict our excess return signals on a security level. These signals are absolute. </Header>

In [13]:
#First, import our packages for the database connection and dataframe access
import mysql.connector
import pandas as pd
import statsmodels.api as sm
import numpy as np
import warnings
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [14]:
#Ignore warnings and set max row display option
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 300)
pd.set_option('display.min_rows',300)

In [15]:
# Connect to MySQL database
connection = mysql.connector.connect(
    host="ubctg.con7266gcvin.us-east-2.rds.amazonaws.com",
    user="admin",
    password="ubctgquant",
    database="ubctg"
)

Here, we pull monthly returns across the stock universe over 10 years from 2011 to 2021

In [16]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2018-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT * FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}'"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
universe_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
universe_df = pd.DataFrame(universe_data, columns=columns)

#close cursor and db connection
cursor.close()

True

Generate the number of observations per security in our table. This way we can remove those that do not have a full dataset

In [17]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2017-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT COUNT(PERMNO), PERMNO FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}' GROUP BY PERMNO"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
observationTable_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
observation_df = pd.DataFrame(observationTable_data, columns=columns)

#close cursor and db connection
cursor.close()

True

Here, we remove securities that do not have enough observations (in our case, we look for at least 120)

In [18]:
#Filter securities with at least median observations in the period. This is not a great way to eliminate observations, so I would be cautious in doing this. There are better ways to filter out insufficient data
observation_df_filtered = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >= int(pd.to_numeric(observation_df['COUNT(PERMNO)']).median())]

#Inner join our dataframes to only keep the securities we have data on
universe_df_filtered = pd.merge(universe_df, observation_df_filtered, on='PERMNO', how='inner')

We can now add a few factors to our model, in this case we will use GDP, CPI (inflation data), and the unemployment rate in each month. We then add it to our monthly returns dataframe for our OLS regression

In [19]:
#Introduce our factor data below as a CSV (pandas datareader not currently working for FRED api)
macro_factors = pd.read_csv("UBCTG Factor Model Example - Macro FRED Data.csv")

#These dates are at the beginning of month, so we will operate on our monthly return dataframe to convert our dates to the beginning of the month so that we can append on index
universe_df_filtered["date"] = pd.to_datetime(universe_df_filtered["date"]).dt.to_period('M').dt.to_timestamp()

#Convert macro factors date column to type datetime64 and drop the non-datetime64 column
macro_factors["date"] = pd.to_datetime(macro_factors["DATE"])
macro_factors = macro_factors.drop('DATE', axis=1)

#Inner-join macro factors dataframe with universe dataframe, using 'date' column as index
universe_df_with_external_factors = pd.merge(universe_df_filtered, macro_factors, on= 'date', how='inner')

#Ensure no errors in the returns column (there have been some instances where returns have taken on non-numeric values)
universe_df_with_external_factors["RET"] = pd.to_numeric(universe_df_with_external_factors["RET"], errors="coerce")

#Here, we select our required columns in proper order (see function documentation for more info) and get rid of extra columns that we will not use
universe_df_with_external_factors_filtered = universe_df_with_external_factors[["PERMNO","date","RET", "Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted","CPI (USACPALTT01CTGYM)","UNRATE"]]

#Remove duplicate rows from universe (monthly returns) dataframe to avoid faulty data 
universe_df_with_external_factors_filtered = universe_df_with_external_factors_filtered.drop_duplicates()

universe_df_with_external_factors_filtered.head()

,PERMNO,date,RET,"Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted",CPI (USACPALTT01CTGYM),UNRATE
0,10026,2018-12-01,-0.075094,1.472009,1.949292,3.9
1,10026,2019-01-01,0.067501,2.562449,1.593904,4.0
2,10026,2019-02-01,0.006025,2.696856,1.576422,3.8
3,10026,2019-03-01,0.026146,3.209721,1.933697,3.8
4,10026,2019-04-01,-0.010451,2.545829,2.099331,3.7


In [20]:
import yfinance as yf
from datetime import datetime

start_date='2023-09-16'
# Download Bitcoin price data from Yahoo Finance (data available as far back as 2014-09-16)
btc_df = yf.download('BTC-USD', start=start_date, end=datetime.now(), interval='1d')

# Function to calculate 7-day ROI from Monday to Monday to follow tournament's Monday to Monday forecast schedule.

def calculate_monday_to_monday_roi(btc_df):
    monday_close = btc_df[btc_df.index.weekday == 0]['Close']
    next_monday_close = monday_close.shift(-1)  # Shift by 1 week
    weekly_returns = pd.DataFrame({'Monday_Close': monday_close, 'Next_Monday_Close': next_monday_close})
    weekly_returns.dropna(inplace=True)
    weekly_returns['7_Day_ROI'] = (weekly_returns['Next_Monday_Close'] - weekly_returns['Monday_Close']) / weekly_returns['Monday_Close'] * 100
    weekly_returns = weekly_returns[['7_Day_ROI']]  # Only keep the 7_Day_ROI
    return weekly_returns

# Get the weekly BTC ROI data
weekly_btc_df = calculate_monday_to_monday_roi(btc_df)


AAPL_df = yf.download('AAPL', start=start_date, end=datetime.now(), interval='1d')
weekly_AAPL_df = calculate_monday_to_monday_roi(AAPL_df)



TSLA_df = yf.download('TSLA', start=start_date, end=datetime.now(), interval='1d')
weekly_TSLA_df = calculate_monday_to_monday_roi(TSLA_df)
weekly_TSLA_df.head()


NVDA_df = yf.download('NVDA', start=start_date, end=datetime.now(), interval='1d')
weekly_NVDA_df = calculate_monday_to_monday_roi(NVDA_df)
weekly_NVDA_df.head()

AMD_df = yf.download('AMD', start=start_date, end=datetime.now(), interval='1d')
weekly_AMD_df = calculate_monday_to_monday_roi(AMD_df)
weekly_AMD_df.head()


GOOG_df = yf.download('GOOG', start=start_date, end=datetime.now(), interval='1d')
weekly_GOOG_df = calculate_monday_to_monday_roi(AMD_df)
weekly_GOOG_df.head()

weekly_AAPL_df.rename(columns={'7_Day_ROI': 'AAPL_7_Day_ROI'}, inplace=True)
weekly_TSLA_df.rename(columns={'7_Day_ROI': 'TSLA_7_Day_ROI'}, inplace=True)
weekly_AMD_df.rename(columns={'7_Day_ROI': 'AMD_7_Day_ROI'}, inplace=True)
weekly_NVDA_df.rename(columns={'7_Day_ROI': 'NVDA_7_Day_ROI'}, inplace=True)
weekly_GOOG_df.rename(columns={'7_Day_ROI': 'GOOG_7_Day_ROI'}, inplace=True)


combined_df = pd.concat([weekly_btc_df, weekly_AAPL_df, weekly_TSLA_df, weekly_NVDA_df, weekly_AMD_df, weekly_GOOG_df], axis=1)
combined_df.reset_index(inplace=True)
combined_df['PERMNO'] = 10001
combined_df.insert(0, 'PERMNO', combined_df.pop('PERMNO'))
combined_df.rename(columns={'7_Day_ROI': 'RET'}, inplace=True)
combined_df.head()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,PERMNO,Date,RET,AAPL_7_Day_ROI,TSLA_7_Day_ROI,NVDA_7_Day_ROI,AMD_7_Day_ROI,GOOG_7_Day_ROI
0,10001,2023-09-18,-1.703655,-1.061976,-6.894599,-3.966700,-4.874480,-4.874480
1,10001,2023-09-25,4.685840,-1.323263,1.866473,6.063193,6.048469,6.048469
2,10001,2023-10-02,0.192122,3.015830,3.207475,1.096417,3.582846,3.582846
3,10001,2023-10-09,3.392546,-0.150849,-2.214355,1.815657,-0.476771,-0.476771
4,10001,2023-10-16,16.012808,-3.200538,-16.477629,-6.768636,-6.058611,-6.058611


In [21]:
# Assuming 'combined_df' is your DataFrame and it has been defined earlier

# List of specific columns to apply NaN fixing
columns_to_fix = ['AAPL_7_Day_ROI', 'TSLA_7_Day_ROI', 'NVDA_7_Day_ROI', 'AMD_7_Day_ROI', 'GOOG_7_Day_ROI']

# Function to find the next non-NaN value in the series
def find_next_non_nan(series, start_index, direction=1):
    if direction == 1:  # Search forwards
        for i in range(start_index + 1, len(series)):
            if not pd.isnull(series[i]):
                return series[i]
    else:  # Search backwards
        for i in range(start_index - 1, -1, -1):
            if not pd.isnull(series[i]):
                return series[i]
    return np.nan  # Return np.nan if no non-NaN value is found

# Apply the NaN fixing logic
for column in columns_to_fix:
    for i in range(len(combined_df[column])):
        if pd.isnull(combined_df[column].iloc[i]):
            # Find the next non-NaN values in both directions
            prev_val = find_next_non_nan(combined_df[column], i, direction=-1)
            next_val = find_next_non_nan(combined_df[column], i, direction=1)
            
            # Calculate the average if both values are not NaN, otherwise use the non-NaN value
            if not pd.isnull(prev_val) and not pd.isnull(next_val):
                combined_df[column].iloc[i] = (prev_val + next_val) / 2
            elif not pd.isnull(prev_val):
                combined_df[column].iloc[i] = prev_val
            elif not pd.isnull(next_val):
                combined_df[column].iloc[i] = next_val
            # If both are NaN, do nothing (or handle as needed)

In [22]:
# To find entries with NaN values in the DataFrame `combined_df`
nan_entries = combined_df[combined_df.isna().any(axis=1)]
nan_entries

,PERMNO,Date,RET,AAPL_7_Day_ROI,TSLA_7_Day_ROI,NVDA_7_Day_ROI,AMD_7_Day_ROI,GOOG_7_Day_ROI


In [23]:
# To find entries with NaN values in the DataFrame `combined_df`
nan_entries = combined_df[combined_df.isna().any(axis=1)]
nan_entries

,PERMNO,Date,RET,AAPL_7_Day_ROI,TSLA_7_Day_ROI,NVDA_7_Day_ROI,AMD_7_Day_ROI,GOOG_7_Day_ROI


In [24]:
universe_df_with_external_factors_filtered=combined_df

In [25]:
#Defining the factor model that takes a dataframe of the required columns (unique identifier, date/index column, returns columns, and a set of factors)
def olsfactormodel(df_attached, lookbackwindow:int, returnsvec:bool):

    #Assign passed dataframe to new dataframe
    df = df_attached.copy(deep=True)
    
    #We generate our list of unique tickers using the column in the unique identifier position (0, or leftmost column)
    uniqueTickerList = df.iloc[:,0].unique()
    
    #Initialize large df to drop results of regression for each security at each regression date
    containerdf = pd.DataFrame()
                
    #Set our lookback window to 24 periods (months in this case). This means that we will run a regression for each period (after the first 24 months) using the previous 24 months as data
    LookBack_Window=lookbackwindow
    
    #We will now create a new set of columns for the Beta and P-value for each of our factors. We will do this by iterating through each factor and columns for the beta coefficients
    factorlist = list(df.columns[3:])
    
    #For each factor, define an empty column to hold the corresponding coefficient
    for factor in factorlist: 
        df["Beta_" + factor] = 0


    #For each factor, define an empty column to hold the corresponding t-stat/p-value
    for factor in factorlist: 
        df["P-Value_" + factor] = 0
    
    #Initialize global parameter(s) regardless of factor count
    df["R_squared"] = 0
    df["Constant B0"] = 0

    #For each identifier (ticker), generate a dataframe from the broader dataframe that 
    for ticker in uniqueTickerList:
        
        #For each unique identifier (ticker), we create a dataframe with observations from that particular identifier
        ticker_specific_universe_df = df.loc[df.iloc[:,0] == ticker]
        
        #We then sort our date/index column in position 1 to ensure our date is ascending from the earliest available
        ticker_specific_universe_df = ticker_specific_universe_df.sort_values(by= ticker_specific_universe_df.columns[1])

        #Shift our returns data back by one period. This way, we regress "t" factors to "t+1" returns, and our betas become forecasts
        ticker_specific_universe_df['RET'] = ticker_specific_universe_df['RET'].shift(-1)

        if returnsvec == True:
            ticker_specific_universe_df = ticker_specific_universe_df.tail(LookBack_Window+1)

        #For each lookback window span, train an OLS and collect the results
        for x in range(0, (len(ticker_specific_universe_df)-LookBack_Window)):
            
            # Define the independent variables (X) and dependent variable (Y). X's are defined by our factor columns and our returns are defined in our third left-most column (position 2)
            X = ticker_specific_universe_df[factorlist][x:x+LookBack_Window]
            Y = ticker_specific_universe_df[ticker_specific_universe_df.columns[2]][x:x+LookBack_Window]
    
            #Add a constant term to the independent variables, check impact
            X = sm.add_constant(X)
        
            #Fit the linear regression model
            model = sm.OLS(Y, X)
            results = model.fit()
            
            #Place our regression coefficients into their appropriate columns
            ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], "Constant B0"] = results.params[0]
            ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], "R_squared"] = results.rsquared
    
            #We do the same dynamically using our list of factors and placing into the appropriate factor column
            #Define an index (starting position for inserting regression params)
            i = 3 + len(factorlist) - 1
            
            for index in range(len(factorlist)):
                #increase our indices by one for each factor
                index+=1
                i+=1
                j = i+len(factorlist)
                
                ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], ticker_specific_universe_df.columns[i]] = results.params[index]
                ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], ticker_specific_universe_df.columns[j]] = results.pvalues[index]

        #Add ticker dataframe to larger container dataframe
        if returnsvec==True:
            containerdf = pd.concat([containerdf, ticker_specific_universe_df[ticker_specific_universe_df["date"]==np.max(ticker_specific_universe_df.date)]], ignore_index=True)
        else:
            containerdf = pd.concat([containerdf, ticker_specific_universe_df], ignore_index=True)
    return containerdf

In [26]:
returnsdf = olsfactormodel(universe_df_with_external_factors_filtered,24,False)
returnsdf.head()

,PERMNO,Date,RET,AAPL_7_Day_ROI,TSLA_7_Day_ROI,NVDA_7_Day_ROI,AMD_7_Day_ROI,GOOG_7_Day_ROI,Beta_AAPL_7_Day_ROI,Beta_TSLA_7_Day_ROI,Beta_NVDA_7_Day_ROI,Beta_AMD_7_Day_ROI,Beta_GOOG_7_Day_ROI,P-Value_AAPL_7_Day_ROI,P-Value_TSLA_7_Day_ROI,P-Value_NVDA_7_Day_ROI,P-Value_AMD_7_Day_ROI,P-Value_GOOG_7_Day_ROI,R_squared,Constant B0
0,10001,2023-09-18,4.685840,-1.061976,-6.894599,-3.966700,-4.874480,-4.874480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,2023-09-25,0.192122,-1.323263,1.866473,6.063193,6.048469,6.048469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10001,2023-10-02,3.392546,3.015830,3.207475,1.096417,3.582846,3.582846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10001,2023-10-09,16.012808,-0.150849,-2.214355,1.815657,-0.476771,-0.476771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10001,2023-10-16,4.280115,-3.200538,-16.477629,-6.768636,-6.058611,-6.058611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
returnsdf.tail()

,PERMNO,Date,RET,AAPL_7_Day_ROI,TSLA_7_Day_ROI,NVDA_7_Day_ROI,AMD_7_Day_ROI,GOOG_7_Day_ROI,Beta_AAPL_7_Day_ROI,Beta_TSLA_7_Day_ROI,Beta_NVDA_7_Day_ROI,Beta_AMD_7_Day_ROI,Beta_GOOG_7_Day_ROI,P-Value_AAPL_7_Day_ROI,P-Value_TSLA_7_Day_ROI,P-Value_NVDA_7_Day_ROI,P-Value_AMD_7_Day_ROI,P-Value_GOOG_7_Day_ROI,R_squared,Constant B0
37,10001,2024-06-03,-4.347411,-0.469002,-1.418118,5.904349,-1.962707,-1.962707,-0.853345,0.045126,0.167294,0.031774,0.031774,0.224069,0.816697,0.600490,0.819612,0.819612,0.088191,1.247480
38,10001,2024-06-10,-9.344044,12.194492,7.854312,7.545771,-1.209930,-1.209930,-1.029815,0.048000,0.148194,0.057657,0.057657,0.182788,0.806107,0.643031,0.680509,0.680509,0.104654,1.228025
39,10001,2024-06-17,4.271196,-3.936862,-2.592830,-9.825924,1.167933,1.167933,-0.996568,0.056431,0.143678,0.048532,0.048532,0.082655,0.774311,0.651945,0.730370,0.730370,0.160947,1.194013
40,10001,2024-06-24,-9.779935,4.136639,14.941395,5.240879,-1.597502,-1.597502,-0.920044,-0.010879,0.166745,0.101363,0.101363,0.087977,0.954644,0.542914,0.458082,0.458082,0.190902,1.298581
41,10001,2024-07-01,NaN,5.107270,20.527972,3.137565,13.317268,13.317268,-0.682838,-0.192182,0.235378,0.204336,0.204336,0.199494,0.326062,0.373171,0.158303,0.158303,0.291285,0.816066
